<a href="https://colab.research.google.com/github/Qu1nnD/CS290/blob/main/TeamProject2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import CategoricalNB, GaussianNB

In [8]:
shrooms = pd.read_csv("https://raw.githubusercontent.com/Qu1nnD/CS290/main/data/mushroom_cleaned.csv")
shrooms.describe()

,cap-diameter,cap-shape,gill-attachment,gill-color,stem-height,stem-width,stem-color,season,class
count,54035.000000,54035.000000,54035.000000,54035.000000,54035.000000,54035.000000,54035.000000,54035.000000,54035.000000
mean,567.257204,4.000315,2.142056,7.329509,0.759110,1051.081299,8.418062,0.952163,0.549181
std,359.883763,2.160505,2.228821,3.200266,0.650969,782.056076,3.262078,0.305594,0.497580
min,0.000000,0.000000,0.000000,0.000000,0.000426,0.000000,0.000000,0.027372,0.000000
25%,289.000000,2.000000,0.000000,5.000000,0.270997,421.000000,6.000000,0.888450,0.000000
50%,525.000000,5.000000,1.000000,8.000000,0.593295,923.000000,11.000000,0.943195,1.000000
75%,781.000000,6.000000,4.000000,10.000000,1.054858,1523.000000,11.000000,0.943195,1.000000
max,1891.000000,6.000000,6.000000,11.000000,3.835320,3569.000000,12.000000,1.804273,1.000000


In [9]:
shrooms.head()

,cap-diameter,cap-shape,gill-attachment,gill-color,stem-height,stem-width,stem-color,season,class
0,1372,2,2,10,3.807467,1545,11,1.804273,1
1,1461,2,2,10,3.807467,1557,11,1.804273,1
2,1371,2,2,10,3.612496,1566,11,1.804273,1
3,1261,6,2,10,3.787572,1566,11,1.804273,1
4,1305,6,2,10,3.711971,1464,11,0.943195,1


In [ ]:
shrooms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54035 entries, 0 to 54034
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   cap-diameter     54035 non-null  int64  
 1   cap-shape        54035 non-null  int64  
 2   gill-attachment  54035 non-null  int64  
 3   gill-color       54035 non-null  int64  
 4   stem-height      54035 non-null  float64
 5   stem-width       54035 non-null  int64  
 6   stem-color       54035 non-null  int64  
 7   season           54035 non-null  float64
 8   class            54035 non-null  int64  
dtypes: float64(2), int64(7)
memory usage: 3.7 MB


In [10]:
features = shrooms.drop(columns=['class'])
target = shrooms['class']
poison_instance = {'cap-diameter': 1372, 'cap-shape': 2, 'gill-attachment': 2, 'gill-color': 10, 'stem-height': 3.807, 'stem-width':1545 , 'stem-color': 11}
non_poison_instance = {'cap-diameter': 914, 'cap-shape': 2, 'gill-attachment': 0, 'gill-color': 10, 'stem-height': 1.47, 'stem-width':1418 , 'stem-color': 11}
new_instance1 = {'cap-diameter': 1000, 'cap-shape': 2, 'gill-attachment': 1, 'gill-color': 10, 'stem-height': 2.5, 'stem-width':1200 , 'stem-color': 11}
new_instance2 = {'cap-diameter': 950, 'cap-shape': 2, 'gill-attachment': 0, 'gill-color': 10, 'stem-height': 2.35, 'stem-width':1400 , 'stem-color': 7}

In [15]:
def naive_bayes_classifier(features, target, new_instance):
    classes = target.unique()
    priors = {}
    likelihoods_mean = []
    likelihoods_probs = []
    for cls in classes:
        priors[cls] = len(target[target == cls]) / len(target)
        features_cls = features[target == cls]

        for col in features.columns:
            if features[col].nunique() < 20:
                feature_probs = features_cls[col].value_counts(normalize=True)
                likelihoods_probs.append((col, cls, feature_probs))
            else:
                mean = features_cls[col].mean()
                std = features_cls[col].std()
                likelihoods_mean.append((col, cls, mean, std))
    probs_dict = {}
    for cls in classes:
        probs = np.log(priors[cls])
        for feature, value in new_instance.items():
            likelihood = 1e-6
            found = False

            for col, cls_likelihood, probs_array in likelihoods_probs:
                if feature == col and cls == cls_likelihood:
                    likelihood = probs_array.get(value, 1e-6)
                    found = True

            if not found:
                for col, cls_likelihood, mean, std in likelihoods_mean:
                    if feature == col and cls == cls_likelihood:
                        likelihood = (1 / (np.sqrt(2 * np.pi) * std)) * np.exp(-((value - mean) ** 2) / (2 * std ** 2))
                        found = True

            if found:
                probs += np.log(likelihood)

        probs_dict[cls] = probs

    return max(probs_dict, key=probs_dict.get)

In [16]:
predicted_class1=naive_bayes_classifier(features,target,poison_instance) # Practice to see if it knows that a poison instance is poisonous
predicted_class1

1

In [17]:
predicted_class2=naive_bayes_classifier(features,target,non_poison_instance) # Practice to see if it knows that a non-poison instance is non-poisonous
predicted_class2

0

In [18]:
predicted_class3=naive_bayes_classifier(features,target,new_instance1)
predicted_class3

1

In [19]:
predicted_class4=naive_bayes_classifier(features,target,new_instance2)
predicted_class4

1

In [20]:
gnb = GaussianNB().fit( np.array(features), np.array( shrooms["class"] ).reshape(-1,1))

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
gnb.predict(np.array(new_instance1))

TypeError: float() argument must be a string or a real number, not 'dict'